In [1]:
%matplotlib qt5

In [2]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the directory where dynamics.py and utils.py are located to sys.path
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import rmse
from datetime import datetime

In [3]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": 'Computer Modern Roman'
}) 

In [4]:
# MATLAB default color cycle
matlab_default_colors = [
    [0, 0.4470, 0.7410],    # blue
    [0.8500, 0.3250, 0.0980], # orange
    [0.9290, 0.6940, 0.1250], # yellow
    [0.4940, 0.1840, 0.5560], # purple
    [0.4660, 0.6740, 0.1880], # green
    [0.3010, 0.7450, 0.9330], # light blue
    [0.6350, 0.0780, 0.1840]  # red
]

# Set the color cycle in Matplotlib to match MATLAB's default
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=matlab_default_colors)

In [5]:
def get_latest_file(prefix, suffix, directory='../data_recent/'):
    files = os.listdir(directory)
    # Filter files by prefix and suffix
    filtered_files = [f for f in files if f.startswith(prefix) and f.endswith(suffix)]
    
    # Extract date from filename and sort files by this date
    def extract_date(file_name):
        # Assuming the date is always in the format 'YYYYMMDD_HHMMSS' and is at a fixed position
        # Adjust the slice indices based on your actual filename structure
        date_str = file_name.split('_')[-2] + file_name.split('_')[-1].split('.')[0]
        return datetime.strptime(date_str, "%Y%m%d%H%M%S")
    
    filtered_files.sort(key=lambda x: extract_date(x), reverse=True)
    
    return os.path.join(directory, filtered_files[0]) if filtered_files else None

In [10]:
# Define the prefix, suffix, and directory
prefix = 'unkkt_form2_'
suffix = '.pkl'
directory = '../data_recent/'

# List and filter files based on prefix and suffix
files = os.listdir(directory)
filtered_files = [os.path.join(directory, f) for f in files if f.startswith(prefix) and f.endswith(suffix)]

# Initialize the data structure for merged data
merged_data = {'true': None}
m = 0  # Counter for numeric keys in the merged data

# Iterate through the filtered files and merge the data
for f in filtered_files:
    with open(f, 'rb') as file:
        data = pickle.load(file)
        if merged_data['true'] is None:
            merged_data['true'] = data['true']  # Set 'true' value (assumed to be the same across files)
        for key in data.keys():
            if key != 'true':  # Process keys other than 'true'
                merged_data[m] = data[key]
                m += 1

# Generate the output file name based on the current date and time
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(directory, f"{prefix}{current_time}.pkl")

# Write the merged data to the new pkl file
with open(output_file, 'wb') as file:
    pickle.dump(merged_data, file)

merged_data

{'true': array([[[-1.29558451e+06, -1.73814204e+06, -2.17339750e+06, ...,
          -2.18842751e+06, -2.61419944e+06, -3.02898462e+06]],
 
        [[-9.29374091e+05, -9.16018270e+05, -8.98814218e+05, ...,
          -9.05576969e+05, -8.85911935e+05, -8.62523682e+05]],
 
        [[ 6.79341168e+06,  6.69574551e+06,  6.56987355e+06, ...,
           6.56395880e+06,  6.40926168e+06,  6.22755500e+06]],
 
        ...,
 
        [[ 1.36145516e+03,  1.51395656e+03,  1.66002168e+03, ...,
           1.70612125e+03,  1.84083584e+03,  1.96775731e+03]],
 
        [[ 5.66586155e+03,  5.93220647e+03,  6.17347269e+03, ...,
           6.28636479e+03,  6.48914140e+03,  6.66455561e+03]],
 
        [[-4.81333761e+03, -4.43066879e+03, -4.02946519e+03, ...,
          -3.83005228e+03, -3.40454381e+03, -2.96485363e+03]]]),
 0: array([[[-1.29558448e+06, -1.73814200e+06, -2.17339747e+06, ...,
          -2.18842753e+06, -2.61419945e+06, -3.02898463e+06]],
 
        [[-9.29374080e+05, -9.16018263e+05, -8.98814210e+

In [11]:
filename_prefix = 'unkkt_form2_'

# Load the latest files into DataFrames
with open(get_latest_file(filename_prefix, '.pkl'), 'rb') as f:
    nkkt_data = pickle.load(f)

# Simulation parameters
dt = 60.0  # Time step [s]
T = 395  # Duration [min]
T_RMSE = 0  # Duration for RMSE calculation [min]
time = np.arange(0, T) / dt # Time vector [h]
M = len(nkkt_data) - 1 # Number of Monte Carlo simulations

In [12]:
dev_chief_values = []
dev_deputy1_values = []
dev_deputy2_values = []
dev_deputy3_values = []
rmse_chief_values = []
rmse_deputy1_values = []
rmse_deputy2_values = []
rmse_deputy3_values = []
X_true = nkkt_data['true']
for m in range(M):
    X_est = nkkt_data[m]
    
    # Compute relative deviations
    dev_chief = np.linalg.norm(X_est[:3, :, :] - X_true[:3, :, :], axis=0)
    dev_deputy1 = np.linalg.norm(X_est[6:9, :, :] - X_true[6:9, :, :], axis=0)
    dev_deputy2 = np.linalg.norm(X_est[12:15, :, :] - X_true[12:15, :, :], axis=0)
    dev_deputy3 = np.linalg.norm(X_est[18:21, :, :] - X_true[18:21, :, :], axis=0)

    # Compute RMSEs
    rmse_chief = rmse(X_est[:6, :, T_RMSE:], X_true[:6, :, T_RMSE:])
    rmse_deputy1 = rmse(X_est[6:12, :, T_RMSE:], X_true[6:12, :, T_RMSE:])
    rmse_deputy2 = rmse(X_est[12:18, :, T_RMSE:], X_true[12:18, :, T_RMSE:])
    rmse_deputy3 = rmse(X_est[18:24, :, T_RMSE:], X_true[18:24, :, T_RMSE:])
    
    # Only keep the valid values
    invalid_rmse = 1e2
    if rmse_chief < invalid_rmse and rmse_deputy1 < invalid_rmse and rmse_deputy2 < invalid_rmse and rmse_deputy3 < invalid_rmse:
        dev_chief_values.append(dev_chief)
        dev_deputy1_values.append(dev_deputy1)
        dev_deputy2_values.append(dev_deputy2)
        dev_deputy3_values.append(dev_deputy3)
        rmse_chief_values.append(rmse_chief)
        rmse_deputy1_values.append(rmse_deputy1)
        rmse_deputy2_values.append(rmse_deputy2)
        rmse_deputy3_values.append(rmse_deputy3)
        print(f'For Monte Carlo Run #{m + 1} the RMSEs are:')
        print(f'    - Chief: {rmse_chief} m')
        print(f'    - Deputy 1: {rmse_deputy1} m')
        print(f'    - Deputy 2: {rmse_deputy2} m')
        print(f'    - Deputy 3: {rmse_deputy3} m')
    else:
        print(f"(!!) For Monte Carlo Run #{m + 1} the algorithm diverged with RMSEs:")
        print(f"    - Chief: {rmse_chief} m")
        print(f"    - Deputy 1: {rmse_deputy1} m")
        print(f"    - Deputy 2: {rmse_deputy2} m")
        print(f"    - Deputy 3: {rmse_deputy3} m")

# Compute averages
dev_chief_avg = np.mean(dev_chief_values, axis=0).reshape(-1, 1)
dev_deputy1_avg = np.mean(dev_deputy1_values, axis=0).reshape(-1, 1)
dev_deputy2_avg = np.mean(dev_deputy2_values, axis=0).reshape(-1, 1)
dev_deputy3_avg = np.mean(dev_deputy3_values, axis=0).reshape(-1, 1)
print(f"Average RMSEs:")
print(f"    - Chief: {np.mean(rmse_chief_values)} m")
print(f"    - Deputy 1: {np.mean(rmse_deputy1_values)} m")
print(f"    - Deputy 2: {np.mean(rmse_deputy2_values)} m")
print(f"    - Deputy 3: {np.mean(rmse_deputy3_values)} m")

For Monte Carlo Run #1 the RMSEs are:
    - Chief: 0.03336553329698841 m
    - Deputy 1: 0.4182756202773169 m
    - Deputy 2: 0.35176674536518576 m
    - Deputy 3: 0.44454938338474775 m
For Monte Carlo Run #2 the RMSEs are:
    - Chief: 0.03571961866874735 m
    - Deputy 1: 0.49971747924344645 m
    - Deputy 2: 0.4642787575029325 m
    - Deputy 3: 0.5744201759904007 m
For Monte Carlo Run #3 the RMSEs are:
    - Chief: 0.036275699342607125 m
    - Deputy 1: 0.5165752518935833 m
    - Deputy 2: 0.34570195977592233 m
    - Deputy 3: 0.5183027564998739 m
For Monte Carlo Run #4 the RMSEs are:
    - Chief: 0.03157128546712227 m
    - Deputy 1: 0.404502438995283 m
    - Deputy 2: 0.39165989316913935 m
    - Deputy 3: 0.43737226762386394 m
For Monte Carlo Run #5 the RMSEs are:
    - Chief: 0.03519167134530961 m
    - Deputy 1: 0.3769226533445437 m
    - Deputy 2: 0.3823881715696216 m
    - Deputy 3: 0.4303806105367998 m
For Monte Carlo Run #6 the RMSEs are:
    - Chief: 0.03007544426089425 m
 

In [13]:
# Plot positions based on screen size
fig_width = 2 * 6.4  # in inches
fig_height = 2 * 4.8  # in inches

# Create a 2 by 2 figure
fig, axs = plt.subplots(2, 2, figsize=(fig_width, fig_height))

# Plot 1: Chief
axs[0, 0].plot(time[:len(dev_chief_avg)], dev_chief_avg, '.-')
axs[0, 0].grid(which="major", color="#DDDDDD", zorder=1)
axs[0, 0].grid(which="minor", color="#DDDDDD", linestyle=(0, (1, 3)), zorder=2)
axs[0, 0].grid(True, which="both")
axs[0, 0].set_yscale('log')
axs[0, 0].set_xlabel('$t$ [h]', fontsize=12)
axs[0, 0].set_ylabel('$\\left\\|\\hat{\\mathbf{r}}_k^{\\mathcal{C}_1} - \\mathbf{r}_k^{\\mathcal{C}_1}\\right\\|_\\mathrm{av}$ [m]', fontsize=12)
axs[0, 0].set_ylim([1e-2, 1e3])
axs[0, 0].tick_params(axis='both', which='both', direction='in')
axs[0, 0].set_title('Chief')

# Plot 2: Deputy 1
axs[0, 1].plot(time[:len(dev_deputy1_avg)], dev_deputy1_avg, '.-')
axs[0, 1].grid(which="major", color="#DDDDDD", zorder=1)
axs[0, 1].grid(which="minor", color="#DDDDDD", linestyle=(0, (1, 3)), zorder=2)
axs[0, 1].grid(True, which="both")
axs[0, 1].set_yscale('log')
axs[0, 1].set_xlabel('$t$ [h]', fontsize=12)
axs[0, 1].set_ylabel('$\\left\\|\\hat{\\mathbf{r}}_k^{\\mathcal{D}_1} - \\mathbf{r}_k^{\\mathcal{D}_1}\\right\\|_\\mathrm{av}$ [m]', fontsize=12)
axs[0, 1].set_ylim([1e-1, 1e3])
axs[0, 1].set_title('Deputy 1')

# Plot 3: Deputy 2
axs[1, 0].plot(time[:len(dev_deputy2_avg)], dev_deputy2_avg, '.-')
axs[1, 0].grid(which="major", color="#DDDDDD", zorder=1)
axs[1, 0].grid(which="minor", color="#DDDDDD", linestyle=(0, (1, 3)), zorder=2)
axs[1, 0].grid(True, which="both")
axs[1, 0].set_yscale('log')
axs[1, 0].set_xlabel('$t$ [h]', fontsize=12)
axs[1, 0].set_ylabel('$\\left\\|\\hat{\\mathbf{r}}_k^{\\mathcal{D}_2} - \\mathbf{r}_k^{\\mathcal{D}_2}\\right\\|_\\mathrm{av}$ [m]', fontsize=12)
axs[1, 0].set_ylim([1e-1, 1e3])
axs[1, 0].set_title('Deputy 2')

# Plot 4: Deputy 3
axs[1, 1].plot(time[:len(dev_deputy3_avg)], dev_deputy3_avg, '.-')
axs[1, 1].grid(which="major", color="#DDDDDD", zorder=1)
axs[1, 1].grid(which="minor", color="#DDDDDD", linestyle=(0, (1, 3)), zorder=2)
axs[1, 1].grid(True, which="both")
axs[1, 1].set_yscale('log')
axs[1, 1].set_xlabel('$t$ [h]', fontsize=12)
axs[1, 1].set_ylabel('$\\left\\|\\hat{\\mathbf{r}}_k^{\\mathcal{D}_3} - \\mathbf{r}_k^{\\mathcal{D}_3}\\right\\|_\\mathrm{av}$ [m]', fontsize=12)
axs[1, 1].set_ylim([1e-1, 1e3])
axs[1, 1].set_title('Deputy 3')

fig.suptitle(f'Formation 1', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

2024-12-10 22:29:33.990 python[31722:1344785] +[IMKClient subclass]: chose IMKClient_Legacy
2024-12-10 22:29:33.990 python[31722:1344785] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
